In [6]:
import tensorflow as tf
tf.random.set_seed(36)

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, MaxPooling2D, GlobalMaxPool2D, Conv2D, Conv2DTranspose,concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras

import os
import numpy as np

from sklearn.model_selection import GridSearchCV


In [19]:
#https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47

def conv2d_block(input_tensor, n_filters, kernel_size, batchnorm, activation):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation(activation)(x)
    
    return x
  
def get_unet(input_img, n_filters, dropout, batchnorm, kernel_size, activation, depth):
    # Contracting Path
    c=[None]*(depth+3)
    #form 0(=init) to depth(=last decode), also bottom added as depth[depth+1]
    p=[input_img]*(depth+2)
    #Encode
    for current_depth in range(depth):
        current_depth+=1
        c[current_depth]=conv2d_block(p[current_depth-1], n_filters * pow(2,current_depth-1), kernel_size, batchnorm, activation)
        p[current_depth]=MaxPooling2D((2,2))(c[current_depth])
        p[current_depth]=Dropout(dropout)(p[current_depth])
    #Bottom
    c[depth+1]=conv2d_block(p[depth],n_filters* pow(2,depth),kernel_size,batchnorm, activation)
        
    #Decode
    u=[None]*(depth+2)
    d=[c[depth+1]]*(depth+2)
    for current_depth in reversed(range(depth)):
        u[current_depth] = Conv2DTranspose(n_filters * pow(2,current_depth), (kernel_size, kernel_size), strides = (2, 2), padding = 'same')(d[current_depth+1])
        u[current_depth] = concatenate([u[current_depth], c[current_depth+1]])
        u[current_depth] = Dropout(dropout)(u[current_depth])
        d[current_depth] = conv2d_block(u[current_depth], n_filters * pow(2,current_depth), kernel_size, batchnorm, activation)

    #decision
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d[0])
    model = Model(inputs=[input_img], outputs=[outputs])
    #new
    model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
    model.save('models/'+string+'.h5')
    plot_model(model, to_file='diagrams/'+string+'.png')
    return model

im_height=256
im_width=256
input_img = Input((im_height, im_width, 1), name='img')
#mode = get_unet(input_img, n_filter, dropout, Batchnormalization, convolution window size, activation function, depth)
#standard ->unet(input_img, 16, 0.05,True,3,'relu', 4)
filters=[8,16,32]
dropouts=[5,25,50]
window_sizes=[3,5]
activations=['relu','selu','tanh','sigmoid']
depths=[3,5]
for föuter in filters:
    for dropout in dropouts:
        for window_size in window_sizes:
            for activation in activations:
                for depth in depths:
                    string=str(föuter)+'F'+str(dropout)+'Dr'+str(window_size)+'Ws'+activation[:2]+'A'+str(depth)+'D'
                    model = get_unet(input_img, föuter, dropout/100, True, window_size, activation, depth)
                    model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
                    plot_model(model, to_file='diagrams/'+string+'.png')
                    model.save('models'+str(depth)+'/'+string+'.h5')
                    print('Done with: '+string)

In [9]:
train_images_path='project3_data/data/train_images'
train_labels_path='project3_data/data/train_labels'

for file in os.listdir(train_images_path):
    images=np.load(train_images_path+'/'+file)
    labels=np.load(train_labels_path+'/'+file)
    overall_depth=images.shape[0]
    for curr_depth in range(overall_depth):
        train_images=np.expand_dims(images[curr_depth],axis=2)
        train_labels=np.expand_dims(labels[curr_depth], axis=2)
        
X=np.array(train_images)
print(X.shape)
Y=np.array(train_labels)
print(Y.shape)

(256, 256, 1)
(256, 256, 1)


In [17]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

filters=[8,16,32]
dropouts=[5,25,50]
window_sizes=[3,5]
activations=['relu','selu','tanh','sigmoid']
depths=[3,5]
strings=[]
validation_scores=[]
test_scores=[]
for föuter in filters:
    for dropout in dropouts:
        for window_size in window_sizes:
            for activation in activations:
                for depth in depths:
                    string=str(föuter)+'F'+str(dropout)+'Dr'+str(window_size)+'Ws'+activation[:2]+'A'+str(depth)+'D'
                    model = get_unet(Input((256, 256, 1), name='img'), föuter, dropout/100, True, window_size, activation, depth)
                    print('Painted model: '+string)
                    model.fit(X,Y)
                    validationY=model.predict(validationX)
                    validation_scores.append(scoring(validationY,validation_labels))
                    strings.append(string)
                    testY=model.predict(testX)
                    print(scoring(testY,test_labels))
                    print('Done model: '+string)
lowest_index=validation_scores.index(min(validation_scores))
print('Best Model: '+strings[lowest_index])
best_model=keras.models.load('models/'+strings[lowest_index]+'.h5')
best_model.fit(X100,Y100)
testY=best_model.predict(testX)
print(scoring(testy,test_labels))

Painted model: 8F5Dr3WsreA1D


ValueError: Error when checking input: expected img to have 4 dimensions, but got array with shape (256, 256, 1)

(256, 256, 1)
